# Python для сбора и анализа данных

*Алла Тамбовцева, НИУ ВШЭ*

## Измерение времени исполнения кода

В Python есть два модуля, позволяющие измерить время исполнения кода: `timeit` и `time`. Точнее, первый модуль непосредственно измеряет время исполнения кода, а второй позволяет зафиксировать точное время начала исполнения кода и время его окончания (далее путем нехитрого вычитания можно получить результат). Измерение времени с помощью `timeit` является более точным, поскольку его функции исполняют код много раз и усредняют полученные на каждой итерации результаты. Почему одного «прогона» кода часто недостаточно? Во-первых, время и скорость исполнения кода зависит от состояния и загруженности системы, поэтому, прогнав код один раз, мы получаем «локальный» результат, который может существенно отличаться от результатов, полученных позже. Во-вторых, если код короткий и функции сами по себе очень быстрые, за одно исполнение сложно зафиксировать время работы с высокой точностью. Поэтому, чтобы получить надёжные результаты, имеет смысл прогонять код много раз.

Модуль `timeit` можно импортировать явно, но тогда код, время исполнения которого оценивается, нужно записывать как строку:

In [1]:
import timeit

In [2]:
code = """
def square(x):
    return x ** 2
square(101)
"""

In [3]:
timeit.timeit(code, number=10000)  # number – число прогонов 

0.0042615830006980104

Код выше измеряет время исполнение кода за 10000 раз. Чтобы получить результат для одного прогона кода, нужно поделить результат на 10000. Получим примерно $0.0000004 = 4 *10^{-7}$ или примерно 400 наносекунд (1 наносекунда = $10^{-9}$ секунд). 

Можно воспользоваться функцией `repeat()` и повторить 10000 прогонов кода несколько раз, чтобы потом посмотреть на минимальное и максимальное время исполнения кода за эти несколько повторов, а также на среднее время исполнения кода:

In [4]:
# min, max, average за 3 повтора по 10000 итераций
timeit.repeat(code, repeat=3, number=10000) 

[0.004412323998622014, 0.004582392999509466, 0.0036886689995299093]

Однако, в Jupyter Notebook есть более удобный способ оценивания времени кода, который не предполагает сохранения кода в виде строки. Этот метод использует *магические команды* (это не шутка, они так и назваются *magic commands*, про них можно почитать [здесь](https://ipython.readthedocs.io/en/stable/interactive/magics.html)). Эти команды всегда начинаются с `%` или с `%%` (одинарный символ для строки, двойной – для всей ячейки с кодом) и указываются в начале ячейки с кодом. Они обязательно должны идти отдельной строкой (при добавлении комментария в той же строке, мы получим ошибку). Для примера получим список всех магических команд:

In [5]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %profile  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python  %%python

А теперь вызовем нужный нам *magic*, команду для измерения времени исполнения кода в ячейке.

In [6]:
%%timeit
def square(x):
    return x ** 2
square(101)

337 ns ± 9.88 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Результат выглядит несколько иначе. Сначала указывается среднее время исполнения кода плюс-минус погрешность (стандартное отклонение), а затем следуют пояснения. `Runs` здесь – это число повторений исполнения кода в ячейке, на основе которого считается итоговый результат, а `loops` – это число итераций в рамках одного повторения. Другими словами, процесс измерения времени исполнения кода выглядит так:

- фиксируется число повторений $r$ и число итераций $n$;
- код исполняется $n$ раз, на каждом шаге от $1$ до $n$ фиксируется время исполнения кода, по итогам $n$ прогонов считается среднее время исполнения кода;
- описанная выше операция повторяется $r$ раз, в итоге берется лучший результат (результат того прогона, где среднее время наименьшее).

Получается, что всего код выполняется $n*r$ раз! В примере выше код исполнился 7000000 раз. И наименьшее среднее время исполнения составляет 337 наносекунд (1 наносекунда = $10^{-9}$ секунд). Результаты немного отличаются от того, что мы получали выше в `timeit.timeit()`, но это нормально. Даже усредненные результаты раз от раза будут немного отличаться:

In [7]:
%%timeit
def square(x):
    return x ** 2
square(101)

342 ns ± 3.26 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [8]:
%%timeit
def square(x):
    return x ** 2
square(101)

340 ns ± 7.72 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Но эти отличия не являются значимыми, устройство алгоритмов измерения времени делает возможным сравнения разных реализаций одной и той же операции. Для примера сравним скорость исполнения кода с циклом `for` и кода со списковыми включениями (мы обсуждали ранее, что списковые включения работают быстрее):

In [9]:
%%timeit
L1 = [4, 5, 9, 10, 12, 23, 2]
L2 = []
for i in L1:
    L2.append(i ** 2)

1.94 µs ± 27.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [10]:
%%timeit
L1 = [4, 5, 9, 10, 12, 23, 2]
L2 = [i ** 2 for i in L1]

1.81 µs ± 25.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Действительно быстрее! А теперь попробуем сравнить код с циклом и условиями и более сложными списковыми включениями.

In [11]:
index = [1, 6, 8, 5, 4, 3]

In [12]:
%%timeit
binary = []
for i in index:
    if i >= 3:
        binary.append(1)
    else:
        binary.append(0)

538 ns ± 1.77 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [13]:
%%timeit
binary = [1 if i >=3 else 0 for i in index]

457 ns ± 11 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


И снова списковые включения оказались быстрее. Напоследок сравним время исполнения функции для расчета факториала, написанной своими руками, и функции, встроенной в модуль `math`.

In [14]:
def factorial(n):
    f = 1
    for i in range(1, n+1):
        f = f * i
    return f

In [15]:
%%timeit
factorial(500)

66.6 µs ± 1.62 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [16]:
import math

In [17]:
%%timeit
math.factorial(500)

9.77 µs ± 202 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Разница очень ощутимая (результаты возвращены в милисекундах). Данный пример можно считать иллюстрацией того, что если есть готовая функция Python, на практике лучше использовать её, особенно если она нужна для работы с большим объемом данных.

По умолчанию Python берет очень большое число итераций (оно везде разное, подбирается автоматически в зависимости от сложности кода). Но число итераций можно выставить самостоятельно:

In [18]:
%%timeit -n 100
math.factorial(500)

9.74 µs ± 584 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)
